In [4]:
import sys
import math
import cmath 
from math import sqrt
import numpy as np
import scipy as sp  
from itertools import product
from scipy import special
from scipy.special import iv
from scipy.linalg import sqrtm
from scipy.sparse.linalg import svds, eigs
from numpy import linalg as LA
from numpy.linalg import matrix_power
from numpy import ndarray
import time
import datetime
from opt_einsum import contract

startTime = time.time()
print ("STARTED: " , datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")) 
print ("------------------------")

Niter = 30
Dcut = 35
start = 1.05 
end = 1.07
incr = 0.04
sets = int(((end-start)/incr)+1)  
qstate = 10
Dcut_int = int(Dcut*1.)

L = np.zeros((qstate, qstate), dtype=complex)
out = np.zeros((qstate, qstate,qstate, qstate,qstate, qstate), dtype=complex)
Dcut_triad = Dcut_int


if Niter%3 != 0:
  print("Niter need to be a multiple of 3")
  sys.exit(1)


def dagger(a):
    return np.conjugate(np.transpose(a))

def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")


def tensorsvd(input,left,right,cut):
    
    T = np.transpose(input,left+right)
    left_index_list = []
    for i in range(len(left)):
        left_index_list.append(T.shape[i])
    xsize = np.prod(left_index_list) 
    right_index_list = []
    for i in range(len(left),len(left)+len(right)):
        right_index_list.append(T.shape[i])
    ysize = np.prod(right_index_list)
    T = np.reshape(T,(xsize,ysize))

    U, s, V = sp.linalg.svd(T, full_matrices=False) 
    if cut < len(s):
        s = np.diag(s[:cut])
        U = U[:,:cut]
        V = V[:cut,:]
    else:
        cut = len(s)
        s = np.diag(s)

    U = np.reshape(U,left_index_list+[cut])
    V = np.reshape(V,[cut]+right_index_list)
        
    return U, s, V

def Z3d(beta):

    Wnew = np.zeros((qstate, qstate), dtype=float)
    for i in range (qstate):
        for j in range (qstate):
            if i == j:
                Wnew[i][j] = np.exp(beta)
            else:
                Wnew[i][j] = 1. 
    L = LA.cholesky(Wnew)
    Id = np.eye(qstate)
    A = contract("ax, ay -> xya", L, L)
    B = contract("ab, az -> azb", Id, L)
    C = contract("bc, bz -> bzc", Id, L)
    D = contract("cy, cx -> cyx", L, L)

    return A, B, C, D


def coarse_graining(in1, in2, in3, in4):


    S2 = contract('dze,izj->diej', in2, np.conjugate(in2))
    a = np.shape(S2)[0] * np.shape(S2)[1]
    b = np.shape(S2)[2] * np.shape(S2)[3]
    S2 = np.reshape(S2,(a,b))
    Tmp = contract('fyx,iyx->fi', in4, np.conjugate(in4))
    R2 = contract('ewf,ijk,fk->eiwj', in3, np.conjugate(in3), Tmp)
    a = np.shape(R2)[0] * np.shape(R2)[1]
    b = np.shape(R2)[2] * np.shape(R2)[3]
    R2mat = np.reshape(R2,(a,b))

    S1 = contract('xyd,iyj->xidj', in1, np.conjugate(in1))
    a = np.shape(S1)[0] * np.shape(S1)[1]
    b = np.shape(S1)[2] * np.shape(S1)[3]
    S1 = np.reshape(S1,(a,b))
    Tmp = contract('bizz->bi', R2)
    R3 = contract('awb,ijk,bk->aiwj', in2, np.conjugate(in2), Tmp)
    a = np.shape(R3)[0] * np.shape(R3)[1]
    b = np.shape(R3)[2] * np.shape(R3)[3]
    R3mat = np.reshape(R3,(a,b))

    Kprime = contract('ia,ab,bc,cd,de',S1,S2,R2mat,R3mat.T,S1.T)

    a = int(np.sqrt(np.shape(Kprime)[0]))
    b = int(np.sqrt(np.shape(Kprime)[1]))
    K = np.reshape(Kprime,(b,a,b,a))         
    U, s1, UL = tensorsvd(K,[0,2],[1,3],int(Dcut_int)) 

    S1 = contract('ijk,ipq->jpkq', in1, np.conjugate(in1))
    a = np.shape(S1)[0] * np.shape(S1)[1]
    b = np.shape(S1)[2] * np.shape(S1)[3]
    S1 = np.reshape(S1,(a,b))
    R3 = contract('ijk,pqr,kr->ipjq', in2, np.conjugate(in2), Tmp) 
    a = np.shape(R3)[0] * np.shape(R3)[1]
    b = np.shape(R3)[2] * np.shape(R3)[3]
    R3mat = np.reshape(R3,(a,b))

    Kprime = contract('ia,ab,bc,cd,de',S1,S2,R2mat,R3mat.T,S1.T)

    a = int(np.sqrt(np.shape(Kprime)[0]))
    b = int(np.sqrt(np.shape(Kprime)[1]))
    K = np.reshape(Kprime,(b,a,b,a))
    V, s1, VL = tensorsvd(K,[0,2],[1,3],int(Dcut_int))


    Tmp1 = contract('cqp,pix -> cqix',in4,U)
    Tmp2 = contract('bji,qjy -> biqy',in4,V)
    Tmp3 = contract('cqix,biqy -> cxby',Tmp1,Tmp2)
    MC = contract('ijk,pjr->ipkr', in2, in3)
    Tmp = contract('ijab,azc,cxby->ijyxz', MC, in3, Tmp3)
    G, st, out4 = tensorsvd(Tmp,[0,1,2],[3,4],int(Dcut_int)) 
    G = contract('ijka,al->ijkl', G, st)  
    Tmp1 = contract('pix,pqa->ixqa', np.conjugate(U), in1)
    Tmp2 = contract('qjy,ijd->qyid', np.conjugate(V), in1)
    DC = contract('ixqa,qyid->xayd', Tmp1, Tmp2)
    DC = contract('dzb,xayd->zxyab', in2, DC) 
    Tmp1 = contract('ijkab,abmn->ijkmn', DC, G)
    out1, st2, MCprime = tensorsvd(Tmp1,[0,1],[2,3,4],Dcut) 
    MCprime = contract('ij,jklm->iklm', st2, MCprime)
    out2, st3, out3 = tensorsvd(MCprime,[0,1],[2,3],Dcut)
    sing = sqrtm(st3) 
    out2 = contract('ijk,kp->ijp', out2, sing)
    out3 = contract('kj,jip->kip', sing, out3)

    return out1,out2,out3,out4



if __name__ == "__main__":

    Temp = np.linspace(start, end, num=sets)
    Nsteps = int(np.shape(Temp)[0])
    f = np.zeros(Nsteps)
    mag = np.zeros(Nsteps)

    for p in range (0, Nsteps):

        A, B, C, D  = Z3d(1./Temp[p])
        CU = 0.0 

        for iter in range (Niter):
            
            A, B, C, D = coarse_graining(A,B,C,D)  
            if (iter+1)%3 == 0 and iter > Niter-5:
                Tmp4 = contract('ajb,bjg->ag',B,C)
                Tmp5 = contract('ag,gfi->afi',Tmp4,D)
                Tmp6 = contract('iba,afi->fb',A,Tmp5)
                Xmeas = np.trace(Tmp6) * np.trace(Tmp6)
                Xmeas /= np.trace(Tmp6 @ Tmp6)
                Tmp7 = contract('iba,abj->ij',A,Tmp5)
                Xmeas1 = np.trace(Tmp7) * np.trace(Tmp7)
                Xmeas1 /= np.trace(Tmp7 @ Tmp7)

            norm = LA.norm(A)*LA.norm(B)*LA.norm(C)*LA.norm(D) 
            A /= LA.norm(A)
            B /= LA.norm(B)
            C /= LA.norm(C)
            D /= LA.norm(D)
            CU += np.log(norm)/(2.0**(iter+1))
            
            if iter == Niter-1:
                Tmp1 = contract('dfa,dfj->aj',A,np.conjugate(A))
                Tmp2 = contract('cge,mge->cm',D,np.conjugate(D))
                Tmp3 = contract('ahb,jhk->abjk',B,np.conjugate(B))
                Tmp4 = contract('aj,abjk->bk',Tmp1,Tmp3)
                Tmp5 = contract('bic,kim->bckm',C,np.conjugate(C))
                Z = contract('bckm,bk,cm',Tmp5,Tmp4,Tmp2)
                # Pattern: dfa,ahb,bic,cge,dfj,jhk,kim,mge
                Zpar = CU + (np.log(Z)/(2.0**Niter))
                minus_lnZ = -Zpar.real
                Free = f[p]*(Temp[p])
                print (round(Temp[p],5), -minus_lnZ) 
                file=open("computelnZ_potts.txt", "a+")
                file.write("%4.4f \t  %8.10f  \t %2.0f \t  %2.0f \t %2.0f \t  %8.8f  \n" % (Temp[p], -minus_lnZ, qstate, Niter, Dcut, Xmeas.real))
                file.close()
    
    print ("------------------------")
    print ("COMPLETED: " , datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")) 
    endTime = time.time()
    print ("Running time:", round(endTime - startTime, 2),  "seconds")

STARTED:  2022-01-03 22:44:44
------------------------
1.05 2.893954872055067 0.9999996385060534
------------------------
COMPLETED:  2022-01-03 22:55:37
Running time: 652.52 seconds
